In [451]:
# เริ่มรับข้อมูล
X = ['Klaew, ']
X[:]

['Klaew, ']

In [452]:
### 1. ตัดคำแยกเป็น : LastName, Prefix. name ###
from nltk.tokenize import WhitespaceTokenizer

tokenized_lists = [] # สร้าง list ไว้รอเก็บ เครื่องมือตัดคำ, ค่าตำแหน่ง
for i in range(len(X)):
    tokenized_lists.append((WhitespaceTokenizer(), i)) # เก็บ เครื่องมือตัดคำ, ค่าตำแหน่ง(ไว้ใช้ต่อไป)    
results = []
for tokenized, i in tokenized_lists:
    tokenized_list = tokenized.tokenize(X[i]) 
    results.append(tokenized_list) # เอาผลไว้ใน results

In [453]:
results

[['Klaew,']]

In [454]:
### 2. แยก column เป็น : LastName,Prefix_name. Name
import pandas as pd    
# เก็บข้อมูลลง DataFrame
df_text = pd.DataFrame(results)  # ยังไม่ต้องระบุชื่อคอลัมน์
# การตัดคำจะถูกแยกออกเป็นตัดแล้วได้ 1.ได้คำที่คอลัมน์ 1 พอดี 2. ไปอยู่ที่คอลัมน์ 2 แก้โดยหา idx_fail แล้วเปลี่ยน ณ index นั้นๆ

# กำจัด missing data ด้วยการแทน str : ' '
for i in range(len(df_text.columns)):
    df_text[i] = df_text[i].fillna(' ')

    

In [455]:
# กรณีถ้า คอลมน์ไม่ถึง 4 ให้สร้าง index มาเติมให้ครบ 4
df_text.head()
#for i in range(4):
x = len(df_text.columns)
final = 5
for x in range(x,4):
    print(x)
    df_text[x] = ''

1
2
3


In [456]:
df_text.head()

,0,1,2,3
0,"Klaew,",,,


In [457]:
Prefix_list = ['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Don.', 'Rev.', 'Dr.', 'Mme.', 'Ms.', 'Major.', 'Mlle.', 'Col.', 'Capt.', 'Jonkheer.', 'Sir.', 'Lady.', 'Dona.']
Prefix_special = ''
    
idx_fail = df_text[1].loc[
        
                          (df_text[1] != Prefix_list[0]) & (df_text[1]!= Prefix_list[1]) & (df_text[1] !=Prefix_list[2]) &  (df_text[1] !=Prefix_list[3]) &
                          (df_text[1] != Prefix_list[4]) & (df_text[1]!= Prefix_list[5]) & (df_text[1] !=Prefix_list[6]) &  (df_text[1] !=Prefix_list[7]) &
                          (df_text[1] != Prefix_list[8]) & (df_text[1]!= Prefix_list[9]) & (df_text[1] !=Prefix_list[10]) &  (df_text[1] !=Prefix_list[11]) &
                          (df_text[1] != Prefix_list[12]) & (df_text[1]!= Prefix_list[13]) & (df_text[1] !=Prefix_list[14]) & (df_text[1] !=Prefix_list[15])
        
                          ] 

In [458]:
len(df_text.columns)            

4

In [459]:
# โค้ด python function สำหรับนับพยางค์
import re
def syllables(word) :
    word = word.lower()

    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables

    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately']

    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']

    pre_one = ['preach']

    syls = 0 #added syllable number
    disc = 0 #discarded syllable number

    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls

    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)

    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1

    #3) discard trailing "e", except where ending is "le"  

    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']

    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass

        else :
            disc+=1

    #4) check if consecutive vowels exists, triplets or pairs, count them as one.

    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple

    #5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))

    #6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1

    #7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1

    #8) add one if "y" is surrounded by non-vowels and is not in the last word.

    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1

    #9) if starts with "tri-" or "bi-" and is followed by a vowel, add one.

    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1

    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1

    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"

    if word[-3:] == "ian" : 
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1

    #11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:2] == "co" and word[2] in 'eaoui' :

        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1

    #12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1

    #13) check for "-n't" and cross match with dictionary to add syllable.

    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]

    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass   

    #14) Handling the exceptional words.

    if word in exception_del :
        disc+=1

    if word in exception_add :
        syls+=1     

    # calculate the output
    return numVowels - disc + syls

In [460]:
import numpy as np
from  sklearn.feature_extraction.text import TfidfVectorizer
for i in idx_fail.index:
    
    for j in Prefix_list:
        
                # ตรวจดูข้อมูลแล้วเปลี่ยนให้ถูกต้องตามแต่กรณี
        if (df_text[2].loc[i] == j):
            df_text[0].loc[i] = df_text[0].loc[i]+' '+df_text[1].loc[i] # รวมคอลัมน์ 0, 1
            df_text[1].loc[i] = j                 # แทนที่คอลัมน์ 1 ด้วยคอลัมน์ 2
                    
        elif df_text[2].loc[i] == 'Countess.':
            df_text[1].loc[i] = 'the Countess.' # แทนที่ the ด้วย 'the Countess.'
                    
        elif df_text[3].loc[i] == j:
            df_text[0].loc[i] = df_text[0].loc[i]+' '+df_text[1].loc[i]+' '+ df_text[2].loc[i] # รวมคอลัมน์ 0, 1, 2
            df_text[1].loc[i] = j          # แทนที่คอลัมน์ 2 ด้วยคอลัมน์ 3
                    
for i in df_text.index:
        
    for j in Prefix_list:
                
                # กรณีคอลัมน์ 1 = Prefix_list , คอลัมน์ 2 = Prefix_list
        if (df_text[1].loc[i]==j) & (df_text[2].loc[i]==j):
            df_text[2].loc[i] = ''
            k = 2
            for k in range(len(df_text.loc[i])-3):
                        df_text[2].loc[i] += df_text[k+3].loc[i] + ' '
            
            # กรณีคอลัมน์ 1 = Prefix_list , คอลัมน์ 3 = Prefix_list
        elif (df_text[1].loc[i]==j) & (df_text[2].loc[i]!=j) &(df_text[3].loc[i]==j):
            df_text[2].loc[i] = ''
            df_text[3].loc[i] = ''
            k = 2
            for k in range(len(df_text.loc[i])-2):
                df_text[2].loc[i] += df_text[k+2].loc[i] + ' '
             # ทั่วไป กรณีคอลัมน์ 1 = Prefix_list  
        elif (df_text[1].loc[i]==j) & (df_text[2].loc[i]!=j) &(df_text[3].loc[i]!=j):
            k = 2
            for k in range(len(df_text.loc[i])-3):
                df_text[2].loc[i] += ' ' + df_text[k+3].loc[i]
                
                
    # กรณีคำนำหน้าเป็น 'the Countess.'   
    if (df_text[1].loc[i]== 'the Countess.'):
        k = 2
        df_text[2].loc[i] = ' '
        for k in range(len(df_text.loc[i])-2):
            df_text[2].loc[i] += ' ' + df_text[k+2].loc[i]    
     
    # เก็บไว้ใน DataFrame พร้อมตั้งชื่อคอลัมน์
        # เก็บไว้ใน DataFrame พร้อมตั้งชื่อคอลัมน์
    column_features = ['last_name', 'prefix_name', 'name']
    i = 0    
    for feature in column_features: 
        df_text[feature] = df_text[i]
        i += 1
        
    # เลือก features ที่จะนำมาใช้
    cols = ['last_name', 'prefix_name', 'name']
    X_features = df_text[cols]
    
    # นับจำนวน : word_lastName, syllable_lastName, word_name,syllable_name                            
    
    count_word_lastname = []
    count_syllable_name = []
    count_word_name = []
    tfv = TfidfVectorizer() # ตัดคำ
    
    for i in X_features.index:
        # สำหรับนับจำนวนคำ word_lastName
        text = X_features.last_name[i]
        text_list = [text]
        words = tfv.fit_transform(text_list)
        text_cut = tfv.get_feature_names()
        count_word_lastname.append(len(text_cut))
        X_features['count_word_lastname'] = np.array(count_word_lastname)
    
    count_syllable_lastname = []
    for i in X_features.index:    
        # สำหรับนับพยางค์ syllable_lastName
        count_syllable_lastname.append(syllables(X_features.last_name[i]))
        X_features['count_syllable_lastname'] = np.array(count_syllable_lastname)
    
    for i in X_features.index:
        # สำหรับนับจำนวนคำ word_name
        text = X_features.name[i]
        text_list = [text]
        words = tfv.fit_transform(text_list)
        text_cut = tfv.get_feature_names()
        count_word_lastname.append(len(text_cut)) 
        X_features['count_word_name'] = np.array(count_word_name)
    
    for i in X_features.index:
        # สำหรับนับพยางค์ syllable_lastName
        count_syllable_name.append(syllables(X_features.name[i]))
        X_features['count_syllable_name'] = np.array(count_syllable_name)
    

C:\Users\Chaiwat\anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Chaiwat\anaconda3\lib\site-packages\ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
for i in range(2):
    print(i)

In [ ]:
def plus(a,b):
    x = a+b
    return x

def multiple(x):
    x = plus(2,x)*2
    return x

multiple(10)
    

In [ ]:
print(multiple(10))